Working draft for ease of experimenting

In [1]:
## Load environment variables
import os
from dotenv import load_dotenv

load_dotenv()
assert os.environ['LANGCHAIN_API_KEY'], "Please set the LANGCHAIN_API_KEY environment variable"
assert os.environ['GROQ_API_KEY'], "Please set the GROQ_API_KEY environment variable"
assert os.environ['OPENAI_API_KEY'], "Please set the OPENAI_API_KEY environment variable"

DATA_DIR = "../../data/raw"
DATA_CSV_PATH = DATA_DIR + "/data.csv"

Convert raw CSV data to SQL file

In [2]:
## Import data
import pandas as pd

df = pd.read_csv(DATA_CSV_PATH)

In [3]:
## Convert CSV file to SQL file
from sqlalchemy import create_engine, MetaData, Table, text

sqlite_db_path = DATA_DIR + "/orders.db"
engine = create_engine(f"sqlite:///{sqlite_db_path}")

with engine.connect() as conn:
    rows = df.to_sql(name="orders", con=engine, if_exists="replace", index=False)
    print(f"Inserted {rows} rows into the orders table")

    ## Check database
    table = Table('orders', MetaData(), autoload_with=engine)
    print(f"Columns in table '{table.name}':")
    print(table.columns.values())

    rows = conn.execute(text("SELECT * FROM orders LIMIT 5")).fetchall()
    print(f"Sample rows in table '{table.name}':")
    for row in rows:
        print(row)

engine.dispose()

Inserted 5000 rows into the orders table
Columns in table 'orders':
[Column('Instance', TEXT(), table=<orders>), Column('OrderNo', BIGINT(), table=<orders>), Column('ParentOrderNo', BIGINT(), table=<orders>), Column('RootParentOrderNo', BIGINT(), table=<orders>), Column('CreateDate', TEXT(), table=<orders>), Column('DeleteDate', TEXT(), table=<orders>), Column('AccID', BIGINT(), table=<orders>), Column('AccCode', TEXT(), table=<orders>), Column('BuySell', TEXT(), table=<orders>), Column('Side', BIGINT(), table=<orders>), Column('OrderSide', TEXT(), table=<orders>), Column('SecID', BIGINT(), table=<orders>), Column('SecCode', TEXT(), table=<orders>), Column('Exchange', TEXT(), table=<orders>), Column('Destination', TEXT(), table=<orders>), Column('Quantity', BIGINT(), table=<orders>), Column('PriceMultiplier', FLOAT(), table=<orders>), Column('Price', FLOAT(), table=<orders>), Column('Value', FLOAT(), table=<orders>), Column('ValueMultiplier', FLOAT(), table=<orders>), Column('DoneVolum

In [4]:
## Pre-process data - Convert dates to datetime objects
from datetime import datetime

df['CreateDate'] = df['CreateDate'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M"))
df['DeleteDate'] = df['DeleteDate'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M"))

In [5]:
## Get relevant metrics
from utils import calculations

bi1_overview, bi2_sector, bi3_capacity, bi4_lifetime, bi5_price_instruction, \
    bi6_trades_over_time, bi7_sankey_diagram = calculations.calculate_metrics(df)

str(bi1_overview)

"{'current': {'total_number_of_orders': 5000, 'total_volume_of_orders': 20176480, 'total_value_of_orders': 80582268.83}, 'average_historical': {'total_number_of_orders': 5000, 'total_volume_of_orders': 20176480, 'total_value_of_orders': 80582268.83}, 'percentage_difference': {'in_total_orders': 0.0, 'in_total_volume': 0.0, 'in_total_value': 0.0}}"

Initialize LLMs to be used (eg. Groq / Llama, OpenAI)

In [6]:
## Initialize LLM and SQL toolkit
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

groq_llm = ChatGroq(model="llama3-8b-8192")
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
print(db.get_usable_table_names())

toolkit = SQLDatabaseToolkit(db=db, llm=openai_llm)
tools = toolkit.get_tools()
print(tools)

['orders']
[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001EFCDF7E0D0>), InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001EFCDF7E0D0>), ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001EFCDF7E0D0>), QuerySQLCheckerTool(description='Use this 

Create SQL agent that interprets SQL file by generating SQL queries in the background

In [7]:
## Initialize agent
from langgraph.prebuilt import create_react_agent
from langchain import hub

## Prompt template for SQL agent, which contains rules for generating required SQL queries
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
system_message = prompt_template.format(dialect="SQLite", top_k=5)
print(system_message)

agent_executor = create_react_agent(
    model=openai_llm, tools=tools, state_modifier=system_message
)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

In [8]:
## Testing out the SQL agent
from utils import prompts

# example_prompt = "You are working for a financial institution. You are tasked with compliance reporting for the orders of the day. Identify anomalies and provide reasons they are suspicious."
example_prompt = prompts.system_prompt + prompts.order_summary_prompt + """
Based on the above instructions, extract and compile the necessary data from the orders table.
"""


events = agent_executor.stream(
    {"messages": [("user", example_prompt)]},
    stream_mode="values",
)
for event in events:
    curr_message = event["messages"][-1]
    curr_message.pretty_print()

================================ Human Message =================================


You are working at an Australian financial institution and tasked to generate a report for end-of-day reporting.
The report includes business intelligence as well as regulatory compliance information, linked to Australian Securities & Investment Commission (ASIC), which serves to provide clients with insights into their order activity for the day, 
and alert them to possible compliance issues with ASIC.

Generate a trade summary for the end-of-day report. Take note that the ASIC fields are: "OrderGiver", "OrderTakerUserCode", "IntermediaryID", "OriginOfOrder", OrderCapacity", DirectedWholesale", "ExecutionVenue". The flow of the order summary report should be as followed:
1. This should start off with an overview of Orders placed, the total volume, and the total value for the day. 
2. Break down the buy/sell orders for each security, highlighting the buy to sell counts for the top 3 securities. You can a

In [9]:
print(curr_message)

content="Here's the trade summary for the end-of-day report based on the extracted data:\n\n### 1. Overview of Orders Placed\n- **Total Orders**: 5000\n- **Total Volume**: 40,769,413 units\n- **Total Value**: $8,058,226,882.62 AUD\n\n### 2. Breakdown of Buy/Sell Orders for Each Security\n- **Top 3 Securities**:\n  1. **CBA**: \n     - Type: Buy\n     - Orders: 342\n     - Volume: 2,949,152 units\n  2. **JBH**: \n     - Type: Sell\n     - Orders: 340\n     - Volume: 2,824,362 units\n  3. **ACDC**: \n     - Type: Sell\n     - Orders: 336\n     - Volume: 2,631,665 units\n\n### 3. ASIC Reporting Fields Summary\n- **Top 3 Order Givers**:\n  1. **robtest**: 2567 orders\n  2. **robbotest**: 2433 orders\n\n- **Lowest 3 Order Givers**:\n  1. **robbotest**: 2433 orders\n  2. **robtest**: 2567 orders\n\n- **Top 3 Order Taker User Codes**:\n  1. **admin@ROBBOTEST**: 2539 orders\n  2. **admin@IRETEST**: 2461 orders\n\n- **Lowest 3 Order Taker User Codes**:\n  1. **admin@IRETEST**: 2461 orders\n  2.

In [21]:
## Test if LLM can interpret calculated metrics
from langchain_core.output_parsers import StrOutputParser

metrics_prompt = """
Below is an overview of a financial institution's orders for the day.
Specifically, it is a sector-level breakdown.
Please provide a short report of the data.
Data:
"""
# bi2_sector['BHP']['average_historical']['number_of_buy_orders'] = 500
# bi2_sector['BHP']['average_historical']['percentage_of_buy_orders'] = 70
# bi2_sector['BHP']['percentage_difference']['in_number_of_buy_orders'] = 30.5
# bi2_sector['BHP']['percentage_difference']['in_percentage_of_buy_orders'] = 30.5
metrics_prompt = metrics_prompt + str(bi7_sankey_diagram)
print("Prompt: \n", metrics_prompt)

output = openai_llm.invoke(metrics_prompt)
print(StrOutputParser().parse(output.content))

Prompt: 
 
Below is an overview of a financial institution's orders for the day.
Specifically, it is a sector-level breakdown.
Please provide a short report of the data.
Data:
{'BHP': {'percentage_of_BHP_accounts_in_total_orders': 12.7, 'percentage_of_BHP_accounts_in_buy_orders': 12.95, 'percentage_of_BHP_accounts_in_sell_orders': 12.45}, 'NXT': {'percentage_of_NXT_accounts_in_total_orders': 12.02, 'percentage_of_NXT_accounts_in_buy_orders': 11.78, 'percentage_of_NXT_accounts_in_sell_orders': 12.25}, 'PEN': {'percentage_of_PEN_accounts_in_total_orders': 12.32, 'percentage_of_PEN_accounts_in_buy_orders': 12.63, 'percentage_of_PEN_accounts_in_sell_orders': 12.01}, 'CBA': {'percentage_of_CBA_accounts_in_total_orders': 13.28, 'percentage_of_CBA_accounts_in_buy_orders': 13.8, 'percentage_of_CBA_accounts_in_sell_orders': 12.77}, 'IRE': {'percentage_of_IRE_accounts_in_total_orders': 12.4, 'percentage_of_IRE_accounts_in_buy_orders': 12.83, 'percentage_of_IRE_accounts_in_sell_orders': 11.97}, '

Chain: SQL agent --> Report generator LLM

In [ ]:
## Initialize SQL agent
## Imports
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

## Prompt template for SQL agent, which contains rules for generating required SQL queries
sql_prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
sql_system_message = prompt_template.format(dialect="SQLite", top_k=5)

## Configure agent
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
toolkit = SQLDatabaseToolkit(db=db, llm=openai_llm)
tools = toolkit.get_tools()
agent_executor = create_react_agent(
    model=openai_llm, tools=tools, state_modifier=sql_system_message
)

## Helper functions to format agent input and output
##TODO: Replace with Langchain parses and ResponseSchemas
agent_input_helper = RunnableLambda(lambda x: {"messages": x.text})
agent_output_parser = RunnableLambda(lambda x: x["messages"][-1].content)

## Prompt template to specify required metrics
required_insights = """
    total orders placed, total volume, total value of order, breakdown of buy sell for each sector, most frequently ordered sectors, trades created over time, 
    ratio of order capacity, order lifetime breakdown, price instruction breakdown
"""
insight_prompt = PromptTemplate.from_template("From the order data, provide the following insights: {insights}")

## Construct SQL chain
sql_chain = insight_prompt | agent_input_helper | agent_executor | agent_output_parser

## Testing
result = sql_chain.invoke({"insights": required_insights})
print(type(result))
print(result)

In [ ]:
## Initialize report generator LLM
## Imports
from langchain_core.output_parsers import StrOutputParser

## Configure model
openai_llm = ChatOpenAI(model="gpt-4o-mini", temperature=1, api_key=os.environ['OPENAI_API_KEY'])

## Prompt template with sample report
template = """
    You are working at a financial institution and are responsible for end of day reporting.
    Given the insights, generate a report. You may use the following example: 
    {sample_report}
    Write a report based on these insights:
    {insights}
"""
report_prompt = PromptTemplate(
    input_variables=["sample_report", "insights"],
    template=template
)

## Sample report
sample_report = """
    Today, the firm processed a total of 250 orders across multiple sectors, with a combined order volume of 1.5 million shares and an order value of $75 million. Of these, 60% were buy orders, with the remaining 40% being sell orders. The majority of buy orders were concentrated in the Technology and Healthcare sectors, representing approximately 55% of the total order volume, reflecting a continued focus on growth-oriented stocks. Sell orders were predominantly in the Energy and Consumer Goods sectors, reflecting a shift in market sentiment toward risk aversion in those industries.

    The firm handled orders in both agency and principal capacities, with 70% of the trades executed in an agency capacity, ensuring that clients’ interests were prioritized. The remaining 30% were principal trades, where the firm acted as the counterparty. A significant portion of the orders (around 65%) had a short lifetime, executed within the first two hours of placement, indicating high market activity and a swift decision-making process among clients.

    In terms of price instructions, the majority of orders (75%) were executed using limit price instructions, with clients specifying the maximum or minimum prices at which they were willing to trade. The remaining 25% followed market-on-close instructions, aiming to execute trades at the day’s closing price.

    Overall, today’s trading activity was characterized by increased demand in growth sectors, with careful execution through predominantly limit orders and short lifetimes, ensuring optimal timing and price conditions for clients.
"""

## Construct report chain
report_chain = report_prompt | openai_llm | StrOutputParser()

## Testing
##TODO: Find a way to pass the SQL insights to the report chain
report = report_chain.invoke({"sample_report": sample_report, "insights": result})
print(report)


In [ ]:
## Construct chain

Experiment with memory / persistence

In [8]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

## Define a new graph
workflow = StateGraph(state_schema=MessagesState)

## Configure LLM
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

# Define the function that calls the model
def call_model(state: MessagesState):
    response = openai_llm.invoke(state["messages"])
    ## Update message history with response:
    return {"messages": response}


## Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

## Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc123"}}
query_1 = "Hi! I'm Bob. I was born in 2002."
query_2 = "I love basketball, theatre, and coding."

input_messages = [HumanMessage(query_1), HumanMessage(query_2)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! Those are some great interests! Basketball is an exciting sport, theatre is a wonderful way to express creativity, and coding opens up a lot of possibilities in today’s tech-driven world. Do you have a favorite basketball team, a specific type of theatre you enjoy, or a coding project you're currently working on?


In [10]:
state = app.get_state(config).values

for message in state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi! I'm Bob. I was born in 2002.
================================ Human Message =================================

I love basketball, theatre, and coding.
================================== Ai Message ==================================

Hi Bob! Those are some great interests! Basketball is an exciting sport, theatre is a wonderful way to express creativity, and coding opens up a lot of possibilities in today’s tech-driven world. Do you have a favorite basketball team, a specific type of theatre you enjoy, or a coding project you're currently working on?


In [12]:
output = app.invoke({"messages": [HumanMessage("Help me created a profile for a dating app.")]}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Absolutely! Here’s a tailored dating app profile for you:

---

**Profile Name:** Bob, 21

**About Me:**
Hey! I'm Bob, a 21-year-old who loves the thrill of basketball, the magic of theatre, and the creativity of coding. I believe life should be a mix of fun and passion, whether it's shooting hoops with friends, performing on stage, or building something cool with code. 

**Interests:**
- 🏀 Basketball fanatic
- 🎭 Theatre lover (always up for a good show!)
- 💻 Aspiring coder (let’s hack together!)
- 🎶 Music enthusiast (from hip-hop to classical)
- 🌍 Travel junkie (next stop: wherever the adventure takes me)

**Looking For:**
I’m looking to meet someone who’s adventurous, creative, and has a good sense of humor. If you enjoy deep conversations, spontaneous outings, or just chilling with a good movie, let’s connect!

**Fun Fact:**
I once directed a short play and ended up loving it even more than performing!

Output to PDF